In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import glob, os, sys
from pathlib import Path
#import seaborn as sns
import matplotlib._color_data as mcd
import pyarrow.parquet as pq
import click

import matplotlib.dates as mdates
import matplotlib.cbook as cbook
from matplotlib.backends.backend_pdf import PdfPages

import pyarrow.parquet
import pyarrow as pa
import pandas as pd
import Outcomes


import multiprocessing
import pathlib
import time

import click

from SEIR import  NPI, setup, file_paths
from SEIR.utils import config
from SEIR.profile import profile_options
from Outcomes import outcomes

os.chdir('COVIDScenarioPipeline/test/test_inference/')

config_path = 'config.yml'

config.set_file(config_path)

In [2]:
snpi = pq.read_table("model_output/snpi/test_inference/Scenario1/high_death_rate/test_inference/global/intermediate/000000001.000000001.000000001.test_inference.snpi.parquet").to_pandas()
snpi

,geoid,npi_name,start_date,end_date,parameter,reduction
0,15005,Place1,2020-04-01,2020-05-15,r0,0.288982
1,15007,Place1,2020-04-01,2020-05-15,r0,0.199216
2,15009,Place1,2020-04-01,2020-05-15,r0,0.289728
3,15001,Place1,2020-04-01,2020-05-15,r0,0.297097
4,15003,Place1,2020-04-01,2020-05-15,r0,0.286405
5,15005,Place2,"2020-04-01,2020-05-01","2020-04-15,2020-05-15",r0,0.279149
6,15007,Place2,"2020-04-01,2020-05-01","2020-04-15,2020-05-15",r0,0.150002
7,15009,Place2,"2020-04-01,2020-05-01","2020-04-15,2020-05-15",r0,0.201816
8,15001,Place2,"2020-04-01,2020-05-01","2020-04-15,2020-05-15",r0,0.175099
9,15003,Place2,"2020-04-01,2020-05-01","2020-04-15,2020-05-15",r0,0.150430


In [3]:

spatial_config = config["spatial_setup"]
spatial_base_path = pathlib.Path(spatial_config["base_path"].get())
scenario = 'Scenario1'
deathrate = 'high_death_rate'
stoch_traj_flag= True # Truthy: stochastic simulation, Falsy: determnistic mean of the binomial draws
nsim = 10
interactive = False
write_csv = False
write_parquet = True

index = 1
run_id = 'test_inference'
prefix='test_inference/Scenario1/high_death_rate/test_inference/global/intermediate/000000001.'
s = setup.Setup(
    setup_name=config["name"].get() + "_" + str(scenario),
    spatial_setup=setup.SpatialSetup(
        setup_name=spatial_config["setup_name"].get(),
        geodata_file=spatial_base_path / spatial_config["geodata"].get(),
        mobility_file=spatial_base_path / spatial_config["mobility"].get(),
        popnodes_key=spatial_config["popnodes"].get(),
        nodenames_key=spatial_config["nodenames"].get()
    ),
    nsim=nsim,
    npi_scenario=scenario,
    npi_config=config["interventions"]["settings"][scenario],
    seeding_config=config["seeding"],
    ti=config["start_date"].as_date(),
    tf=config["end_date"].as_date(),
    interactive=interactive,
    write_csv=write_csv,
    write_parquet=write_parquet,
    dt=config["dt"].as_number(),
    first_sim_index = index,
    in_run_id = run_id,
    in_prefix = prefix,
    out_run_id = run_id,
    out_prefix = prefix
)

print(f"""
>> Running ***{'STOCHASTIC' if stoch_traj_flag else 'DETERMINISTIC'}*** SEIR and Outcomes modules
>> Scenario: {scenario}
>> Starting {s.nsim} model runs beginning from {s.first_sim_index}
>> Setup *** {s.setup_name} *** from {s.ti} to {s.tf}
>> writing to folder : {s.datadir}{s.setup_name}
    """)
setup_name = s.setup_name


onerun_OUTCOMES_loadID = lambda index: outcomes.onerun_delayframe_outcomes_load_hpar(config,
                                                                                     run_id, prefix, int(index), # input
                                                                                     run_id, prefix, int(index), # output
                                                                                     deathrate, stoch_traj_flag)
onerun_OUTCOMES = lambda index: outcomes.run_delayframe_outcomes(config,
                                                                 run_id, prefix, int(index), # input
                                                                 run_id, prefix, int(index), # output
                                                                 deathrate, nsim=1, n_jobs=1, stoch_traj_flag = stoch_traj_flag)
onerun_SEIR_loadID = lambda sim_id2write, s, sim_id2load: seir.onerun_SEIR_loadID(int(sim_id2write), s, int(sim_id2load), stoch_traj_flag)
onerun_SEIR = lambda sim_id2write, s: seir.onerun_SEIR(int(sim_id2write), s, stoch_traj_flag)

onerun_OUTCOMES(0)
#onerun_OUTCOMES_loadID(0)


>> Running ***STOCHASTIC*** SEIR and Outcomes modules
>> Scenario: Scenario1
>> Starting 10 model runs beginning from 1
>> Setup *** test_inference_Scenario1 *** from 2020-04-01 to 2020-05-15
>> writing to folder : model_output/seir/test_inference/Scenario1/high_death_rate/test_inference/global/intermediatetest_inference_Scenario1
    
Loaded geoids in loaded relative probablity file: 3226 Intersect with seir simulation:  5 keeped
Using 'param_from_file' for relative probability incidI -->  incidH
Using 'param_from_file' for relative probability incidI -->  incidD
Using 'param_from_file' for relative probability incidH -->  incidICU

>> 1 outcomes simulations completed in 0.2 seconds



1

In [4]:
snpi_fn = 'model_output/snpi/test_inference/Scenario1/high_death_rate/test_inference/global/intermediate/000000001.000000001.000000001.test_inference.snpi.parquet'

snpi = pq.read_table(snpi_fn).to_pandas()
npi = NPI.NPIBase.execute(
    npi_config=s.npi_config,
    global_config=config,
    geoids=s.spatset.nodenames,
    loaded_df = snpi)
p_draw = setup.parameters_load(snpi_fn.replace('snpi', 'spar')[:-8],
    'parquet',
    len(s.t_inter),
    s.nnodes
)

#npi = NPI.NPIBase.execute(
#    npi_config=s.npi_config,
#    global_config=config,
#    geoids=s.spatset.nodenames,
#    loaded_df = snpi)

parameters = setup.parameters_reduce(p_draw, npi, s.dt)
beta = parameters[1]

In [5]:
a = snpi[snpi['parameter']=='r0'].parameter.unique()

In [13]:
a = ['sds', 'df'] 

In [15]:
joined_list = [*a, *'r0']
joined_list

['sds', 'df', 'r', '0']

In [16]:
p.be

d
f
s
d
f
